## Задание 1: построить функции принадлежности

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.integrate import odeint 
import ipywidgets as wg 
from IPython.display import display 

n = 100  # количество временных точек для построения
tf = 50.0  # конечное время
SP_start = 2.0  # время изменения уставки

def process(y, t, u):
    Kp = 4.0
    taup = 3.0
    thetap = 1.0
    if t < (thetap + SP_start):
        dydt = 0.0  # временная задержка
    else:
        dydt = (1.0/taup) * (-y + Kp * u)
    return dydt

def pidPlot(Kc, tauI, tauD):
    t = np.linspace(0, tf, n)  # создание временного вектора
    P = np.zeros(n)           # инициализация пропорционального члена
    I = np.zeros(n)           # инициализация интегрального члена
    D = np.zeros(n)           # инициализация дифференциального члена
    e = np.zeros(n)           # инициализация ошибки
    OP = np.zeros(n)          # инициализация выхода регулятора
    PV = np.zeros(n)          # инициализация переменной процесса
    SP = np.zeros(n)          # инициализация уставки
    SP_step = int(SP_start / (tf / (n-1)) + 1)  # начало уставки
    SP[0:SP_step] = 0.0        # задание уставки
    SP[SP_step:n] = 4.0        # увеличение
    y0 = 0.0                  # начальное условие

    for i in range(1, n):
        ts = [t[i-1], t[i]]         # временной интервал
        y = odeint(process, y0, ts, args=(OP[i-1],))  # расчет следующего шага
        y0 = y[1]                   # запись нового начального условия

        PV[i] = y[1]                # запись PV
        e[i] = SP[i] - PV[i]        # вычисление ошибки = SP - PV
        dt = t[i] - t[i-1]          # вычисление шага времени
        P[i] = Kc * e[i]            # вычисление пропорционального члена
        I[i] = I[i-1] + (Kc/tauI) * e[i] * dt  # вычисление интегрального члена
        D[i] = -Kc * tauD * (PV[i]-PV[i-1])/dt  # вычисление дифференциального члена
        OP[i] = P[i] + I[i] + D[i]  # вычисление нового выхода регулятора

    # построение графиков PID-ответа
    plt.figure(1, figsize=(15, 7))
    plt.subplot(2, 2, 1)
    plt.plot(t, SP, 'k-', linewidth=2, label='Setpoint (SP)')
    plt.plot(t, PV, 'r:', linewidth=2, label='Process Variable (PV)')
    plt.legend(loc='best')

    plt.subplot(2, 2, 2)
    plt.plot(t, P, 'g.-', linewidth=2, label=r'Proportional = $K_c \; e(t)$')
    plt.plot(t, I, 'b-', linewidth=2, label=r'Integral = $\frac{K_c}{\tau_I} \int_{i=0}^{n_t} e(t) \; dt $')
    plt.plot(t, D, 'r--', linewidth=2, label=r'Derivative = $-K_c \tau_D \frac{d(PV)}{dt}$')     
    plt.legend(loc='best')

    plt.subplot(2, 2, 3)
    plt.plot(t, e, 'm--', linewidth=2, label='Error (e=SP-PV)')
    plt.legend(loc='best')

    plt.subplot(2, 2, 4)
    plt.plot(t, OP, 'b--', linewidth=2, label='Controller Output (OP)')
    plt.legend(loc='best')
    plt.xlabel('time')

Kc_slide = wg.FloatSlider(value=0.1, min=-0.2, max=1.0, step=0.05)
tauI_slide = wg.FloatSlider(value=4.0, min=0.01, max=5.0, step=0.1)
tauD_slide = wg.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.1)
wg.interact(pidPlot, Kc=Kc_slide, tauI=tauI_slide, tauD=tauD_slide)


interactive(children=(FloatSlider(value=0.1, description='Kc', max=1.0, min=-0.2, step=0.05), FloatSlider(valu…

<function __main__.pidPlot(Kc, tauI, tauD)>

In [7]:
# Import NumPy and scikit-fuzzy 
import numpy as np 
import skfuzzy as fuzz 
 
# Generate universe functions 
ERROR       = np.arange(0, 5, 0.01) 
DELTA_ERROR = np.arange(0, 5, 0.01) 
 
Kc   = np.arange(-0.2, 1.0, 0.01) 
tauI = np.arange(0.01, 5.0, 0.01) 
tauD = np.arange(0.0, 1.0, 0.01) 
 
# Membership functions for ERROR 
SE = fuzz.gaussmf(ERROR, 0.01, 0.8495)
ME = fuzz.gaussmf(ERROR, 2.5, 0.8495)
BE = fuzz.gaussmf(ERROR, 5, 0.8495)
 
# Membership functions for DELTA_EROR 
SDE = fuzz.gaussmf(DELTA_ERROR, 0.01, 0.8495)
MDE = fuzz.gaussmf(DELTA_ERROR, 2.5, 0.8495)
BDE = fuzz.gaussmf(DELTA_ERROR, 5, 0.8495)

# Membership functions for OUTPUT_Kc, OUTPUT_tauI, OUTPUT_tauD
S1 = fuzz.gaussmf(Kc, -0.2, 0.2039) 
M1 = fuzz.gaussmf(Kc, 0.4, 0.2039)
B1 = fuzz.gaussmf(Kc, 1, 0.2039)
 
S2 = fuzz.gaussmf(tauI, 0.01, 0.8476)
M2 = fuzz.gaussmf(tauI, 2.505, 0.8476)
B2 = fuzz.gaussmf(tauI, 5, 0.8476)   
 
S3 = fuzz.gaussmf(tauD, 6.939e-18, 0.1699)
M3 = fuzz.gaussmf(tauD, 0.5, 0.1699)
B3 = fuzz.gaussmf(tauD, 1, 0.1699)
 
def ERROR_category(ERROR_in = 5): 
  ERROR_cat_SMALL   = fuzz.interp_membership(ERROR, SE, ERROR_in)  
  ERROR_cat_MEDIUM  = fuzz.interp_membership(ERROR, ME, ERROR_in)  
  ERROR_cat_BIG     = fuzz.interp_membership(ERROR, BE, ERROR_in)  
  return dict(SMALL_ERROR = ERROR_cat_SMALL,
              MEDIUM_ERROR = ERROR_cat_MEDIUM,
              BIG_ERROR = ERROR_cat_BIG
              ) 

def DELTA_ERROR_category(DELTA_ERROR_in = 5): 
  DELTA_ERROR_cat_SMALL   = fuzz.interp_membership(DELTA_ERROR, SDE, DELTA_ERROR_in)  
  DELTA_ERROR_cat_MEDIUM  = fuzz.interp_membership(DELTA_ERROR, MDE, DELTA_ERROR_in)  
  DELTA_ERROR_cat_BIG     = fuzz.interp_membership(DELTA_ERROR, BDE, DELTA_ERROR_in)  
  return  dict(SMALL_DELTA_ERROR  =  DELTA_ERROR_cat_SMALL,
               MEDIUM_DELTA_ERROR  = DELTA_ERROR_cat_MEDIUM,
               BIG_DELTA_ERROR = DELTA_ERROR_cat_BIG
              ) 
 
ERROR_in = ERROR_category()
DELTA_ERROR_in = DELTA_ERROR_category()

# RULE for OUTPUT_Kc 
rule1 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule2 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule3 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule4 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule5 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule6 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule7 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule8 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule9 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
 
# RULE for tau_I 
rule10 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule11 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR'])
rule12 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule13 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule14 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule15 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule16 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule17 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule18 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
 
# RULE for tau_D 
rule19 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule20 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule21 = np.fmax(ERROR_in['SMALL_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule22 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule23 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule24 = np.fmax(ERROR_in['MEDIUM_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
rule25 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['SMALL_DELTA_ERROR']) 
rule26 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['MEDIUM_DELTA_ERROR']) 
rule27 = np.fmax(ERROR_in['BIG_ERROR'], DELTA_ERROR_in['BIG_DELTA_ERROR']) 
 
# IMPLICATION for Kc 
imp1 = np.fmax(rule1, S1)
imp2 = np.fmax(rule2, S1) 
imp3 = np.fmax(rule3, S1) 
imp4 = np.fmax(rule4, M1) 
imp5 = np.fmax(rule5, M1) 
imp6 = np.fmax(rule6, M1) 
imp7 = np.fmax(rule7, B1) 
imp8 = np.fmax(rule7, B1) 
imp9 = np.fmax(rule7, B1) 
 
# IMPLICATION for tauI 
imp10 = np.fmax(rule10, S2) 
imp11 = np.fmax(rule11, S2) 
imp12 = np.fmax(rule12, S2) 
imp13 = np.fmax(rule13, M2) 
imp14 = np.fmax(rule14, M2) 
imp15 = np.fmax(rule15, M2) 
imp16 = np.fmax(rule16, B2) 
imp17 = np.fmax(rule17, B2) 
imp18 = np.fmax(rule18, B2) 
 
# IMPLICATION for tauD 
imp19 = np.fmax(rule19, S3) 
imp20 = np.fmax(rule20, S3) 
imp21 = np.fmax(rule21, S3) 
imp22 = np.fmax(rule22, M3) 
imp23 = np.fmax(rule23, M3) 
imp24 = np.fmax(rule24, M3) 
imp25 = np.fmax(rule25, B3) 
imp26 = np.fmax(rule26, B3) 
imp27 = np.fmax(rule27, B3) 
 
# Aggregate all output - min 
aggregate_membership1 = np.fmax(imp1, np.fmax(imp2, np.fmax(imp3, np.fmax(imp4, np.fmax(imp5, np.fmax(imp6, np.fmax(imp7, np.fmax(imp8,imp9)))))))) 
aggregate_membership2  =  np.fmax(imp10,  np.fmax(imp11,  np.fmax(imp12,  np.fmax(imp13,  np.fmax(imp14, np.fmax(imp15, np.fmax(imp16, np.fmax(imp17,imp18)))))))) 
aggregate_membership3  =  np.fmax(imp19,  np.fmax(imp20,  np.fmax(imp21,  np.fmax(imp22,  np.fmax(imp23, np.fmax(imp24, np.fmax(imp25, np.fmax(imp26,imp27)))))))) 
 
# Defuzzification 
result_Kc   = fuzz.defuzz(Kc, aggregate_membership1 , 'centroid') 
result_tauI = fuzz.defuzz(tauI, aggregate_membership2 , 'centroid') 
result_tauD = fuzz.defuzz(tauD, aggregate_membership3 , 'centroid') 
print (result_Kc) 
print (result_tauI) 
print (result_tauD)


0.39314757075213463
2.4981591681568136
0.49314447457013927


In [17]:
import numpy as np
import matplotlib.pyplot as plt 
import sys 

num_i_units = 2  # Number of Input units 
num_h_units = 3  # Number of Hidden units 
num_o_units = 3  # Number of Output units 

# The learning rate for Gradient Descent. 
learning_rate = 0.01 
# The parameter to help with overfitting. 
reg_param = 0 
# Maximum iterations for Gradient Descent. 
max_iter = 100 
# Number of training examples 
m = 4 

# Generating the Weights and Biases 
np.random.seed(1) 
W1 = np.random.normal(0, 1, (num_h_units, num_i_units))  # 3x2 
W2 = np.random.normal(0, 1, (num_o_units, num_h_units))  # 3x3 
B1 = np.random.random((num_h_units, 1))  # 3x1 
B2 = np.random.random((num_o_units, 1))  # 3x1 

def sigmoid(z, derv = False): 
    if (derv == True): 
        return z * (1 - z) 
    return 1 / (1 + np.exp(-z))
X = np.array([[0.5, 0.1], [0.8, 0.6], [0.3, 0.9], [0.2, 0.4]])
y = np.array([[0.65], [0.85], [0.45], [0.55]])
coba = np.array([[0.5, 0.1], [0.8, 0.6], [0.3, 0.9], [0.2, 0.4]])



def train(W1, W2, B1, B2):  # Neural Network Training 
    _W1 = np.copy(W1)
    _W2 = np.copy(W2)
    _B1 = np.copy(B1)
    _B2 = np.copy(B2)
    for i in range(max_iter): 
        c = 0 
        dW1 = 0 
        dW2 = 0 
        dB1 = 0 
        dB2 = 0 

        for j in range(m): 
            sys.stdout.write("\rIteration: {} and {}".format(i + 1, j + 1)) 

            # Forward Prop. 
            a0 = X[j].reshape(X[j].shape[0], 1)  # 2x1 

            z1 = _W1.dot(a0) + _B1  # 3x2 * 2x1 + 3x1 = 3x1 
            a1 = sigmoid(z1)  # 3x1 

            z2 = _W2.dot(a1) + _B2  # 3x2 * 2x1 + 3x1 = 3x1 
            a2 = sigmoid(z2)  # 3x1 

            # Back prop. 
            dz2 = a2 - y[j].reshape(y[j].shape[0], 1)  # 3x1 
            dW2 += dz2 * a1.T  # 3x1 .* 1x3 = 3x3 

            dz1 = np.multiply((_W2.T).dot(dz2), sigmoid(a1, derv=True))  # (3x3 * 3x1) .* 3x1 = 3x1 
            dW1 += dz1.dot(a0.T)  # 3x1 * 1x2 = 3x2  

            dB1 += dz1  # 3x1 
            dB2 += dz2  # 3x1 

            c = c + (-(y[j].reshape(y[j].shape[0], 1) * np.log(a2)) - ((1 - y[j].reshape(y[j].shape[0], 1)) * np.log(1 - a2))) 
            sys.stdout.flush()  # Updating the text. 

        _W1 = _W1 - learning_rate * (dW1 / m) + ((reg_param / m) * _W1) 
        _W2 = _W2 - learning_rate * (dW2 / m) + ((reg_param / m) * _W2) 

        _B1 = _B1 - learning_rate * (dB1 / m) 
        _B2 = _B2 - learning_rate * (dB2 / m) 
    return (_W1, _W2, _B1, _B2) 


res = train(W1, W2, B1, B2)

print("\nstart is:\n", (W1, W2, B1, B2), "\n")
print("\nres is:\n", res, "\n")
# Testing 
for j in range(1): 
    a0 = coba[j].reshape(coba[j].shape[0], 1)  # 2x1 
    z1 = W1.dot(a0) + B1  # 3x2 * 2x1 + 3x1 = 3x1 
    a1 = sigmoid(z1)  # 3x1 
    z2 = W2.dot(a1) + B2  # 3x2 * 2x1 + 3x1 = 3x1 
    outNN = sigmoid(z2)  # 3x1 


Iteration: 100 and 4
start is:
 (array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ]]), array([[ 1.74481176, -0.7612069 ,  0.3190391 ],
       [-0.24937038,  1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944]]), array([[0.87638915],
       [0.89460666],
       [0.08504421]]), array([[0.03905478],
       [0.16983042],
       [0.8781425 ]])) 


res is:
 (array([[ 1.61436809, -0.63195272],
       [-0.49670082, -1.05200963],
       [ 0.82800861, -2.32264287]]), array([[ 1.67411951, -0.81654136,  0.29175262],
       [-0.16984999,  1.5097946 , -2.01278008],
       [-0.35638955, -0.41595177,  1.12110061]]), array([[0.8428581 ],
       [0.94954041],
       [0.02172637]]), array([[-0.06240789],
       [ 0.2550807 ],
       [ 0.82329931]])) 

